**2.156 Final Project**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tqdm import tqdm, trange
import seaborn as sns
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load Images 

In [ ]:
images=np.load()

We will want to visualize the data next but will add that in when data is complete

Train test split

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=.2, random_state=42)

In [4]:
class CNN(nn.Module):
    def __init__(self):  # Define all the layers here
        super(CNN, self).__init__()
        # All pooling will use 2x2 window and stride 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # First Conv layer: 3 input channel (RBG), 4 output channels, kernel size 3x3, stride 1, padding 1
        self.conv1 = nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1)

        # Second Conv layer: 4 input channels, 8 output channels
        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)

        # Third Conv layer: 8 input channels, 16 output channels
        self.conv3 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)

        # Fourth Conv layer: 16 input channels, 32 output channels
        self.conv4 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)

        # Operation to flatten the images into a vector
        self.flatten = nn.Flatten()

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 3 * 3, 16)  # Adjusted to match the output size after conv3
        self.fc2 = nn.Linear(16, 75)  # 75 output classes??
        #WILL NEED TO EDIT based on number of parts we decide on

    def forward(self, x):  # Call all the layers in this function
        # Reshape the input to ensure it has the correct channel dimension: [batch x 1 x 50 x 50]
        if len(x.shape) == 3:  # Input is [batch x 50 x 50]
            x = x.unsqueeze(1)  # Add channel dimension -> [batch x 1 x 50 x 50]

        # Convolve, then pass through ReLU, then pooling
        x = F.relu(self.conv1(x))  # [batch x 4 x 50 x 50]
        x = self.pool(x)  # [batch x 4 x 25 x 25]

        x = F.relu(self.conv2(x))  # [batch x 8 x 25 x 25]
        x = self.pool(x)  # [batch x 8 x 12 x 12]

        x = F.relu(self.conv3(x))  # [batch x 16 x 12 x 12]
        x = self.pool(x)  # [batch x 16 x 6 x 6]

        x = F.relu(self.conv4(x))  # [batch x 32 x 6 x 6]
        x = self.pool(x)  # [batch x 32 x 3 x 3]

        x = self.flatten(x)  # Flatten for the fully connected layer [batch x 288]

        # Pass through fully connected layers with ReLU
        x = F.relu(self.fc1(x))  # [batch x 16]

        # Final fully connected layer with no activation (since we are doing regression)
        x = self.fc2(x)  # Output layer with 5 outputs [batch x 5]
        return x

# Create a CNN model instance
model_cnn = CNN()
model_cnn.to(device)

# Print the model's architecture
#summary(model_cnn, input_size=(1, 50, 50))  # Adjusted input size for grayscale images with 1 channel


CNN(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=288, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=75, bias=True)
)

Create Training Loop

In [5]:
def train_model(model, X_train, Y_train, X_validation, Y_validation, num_epochs=10, batch_size=32, learning_rate=0.001, device='cuda'):
    """
    Function to train a given model (CNN or DNN) using training and validation data.

    Args:
    - model: The model to train (e.g., CNN or DNN).
    - X_train: Training data (features).
    - Y_train: Training data (labels).
    - X_validation: Validation data (features).
    - Y_validation: Validation data (labels).
    - num_epochs: Number of training epochs (default: 10).
    - batch_size: Batch size for training (default: 32).
    - learning_rate: Learning rate for the optimizer (default: 0.001).
    - device: Device to run the model on ('cuda' or 'cpu').

    Returns:
    - train_losses: List of training losses for each epoch.
    - val_losses: List of validation losses for each epoch.
    """

    # Move model to the specified device (GPU or CPU)
    model = model.to(device)

    # Define loss function (Mean Squared Error for regression tasks)
    criterion = nn.MSELoss()

    # Define optimizer (Adam optimizer with specified learning rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader for batching the training data
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Create DataLoader for batching the validation data
    validation_dataset = TensorDataset(torch.tensor(X_validation, dtype=torch.float32), torch.tensor(Y_validation, dtype=torch.float32))
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    # Lists to store train and validation losses
    train_losses = []
    val_losses = []

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0

        # Using tqdm for the training loop progress bar
        train_loader_tqdm = tqdm(train_loader, desc=f'Epoch [{epoch + 1}/{num_epochs}]', leave=False)

        # Iterate over batches of training data
        for inputs, labels in train_loader_tqdm:
            inputs = inputs.to(device)  # Move inputs to device (GPU/CPU)
            labels = labels.to(device)  # Move labels to device (GPU/CPU)

            # Zero the gradients before the next update
            optimizer.zero_grad()

            # Forward pass: compute model outputs
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backward pass: compute gradients
            loss.backward()

            # Perform a single optimization step (update model parameters)
            optimizer.step()

            # Accumulate loss over the batch
            running_loss += loss.item() * inputs.size(0)

            # Update tqdm progress bar with current loss
            train_loader_tqdm.set_postfix({'Train Loss': running_loss / len(train_loader.dataset)})

        # Calculate average training loss for the epoch
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)

        # Validation loop (no gradient computation)
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass: compute model outputs
                outputs = model(inputs)

                # Compute validation loss
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

        # Calculate average validation loss for the epoch
        val_loss /= len(validation_loader.dataset)
        val_losses.append(val_loss)

        # Print the train and validation loss at the end of each epoch
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Train MSE: {epoch_loss:.4f} - Validation MSE: {val_loss:.4f}')

    print("Training complete.")

    # Return the recorded training and validation losses
    return train_losses, val_losses


Plot Convergence

In [ ]:
def plot_convergence(train_losses, val_losses):
    """
    Function to plot the training and validation loss convergence over epochs.

    Args:
    - train_losses: List of training losses per epoch.
    - val_losses: List of validation losses per epoch.
    """

    # Create the plot
    plt.figure(figsize=(8, 6))

    # Plot training and validation losses
    plt.plot(train_losses, label='Train Mean Squared Error (MSE)', color="black")
    plt.plot(val_losses, label='Validation Mean Squared Error (MSE)', color="red")

    # Label the axes
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    # Add legend in the upper right corner
    plt.legend(loc='upper right')

    # Remove the top and right borders
    ax = plt.gca()  # Get the current axis
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Show the plot
    plt.show()


Plot Predictions With R-squared: scatterplots to compare our model's predicted values with the ground truth values from the validation set

In [6]:
def plot_predictions_with_r2(model, X_validation, Y_validation, X_test, Y_test, device, names=None, show_test=True):
    """
    Function to plot actual vs predicted values for validation and optionally test datasets,
    display R² scores in the subplot titles, and add a legend in the last (unused) subplot.

    Args:
    - model: The trained model (DNN or CNN) used for making predictions.
    - X_validation: Validation feature data.
    - Y_validation: Validation target data.
    - X_test: Test feature data.
    - Y_test: Test target data.
    - device: The device (GPU or CPU) on which the model is running.
    - names: List of names for the target variables (default: None).
    - show_test: Boolean flag to control whether to show test data or only validation data (default: True).
    """

    if names is None:
        names = ["C11", "C12", "C22", "C66", "Volume Fraction"]  # Default names for target variables

    # Generate predictions for the validation and test sets
    val_pred = model(torch.tensor(X_validation, dtype=torch.float32).to(device)).cpu().detach().numpy()

    if show_test:
        test_pred = model(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().detach().numpy()

    # Create a figure with 2x3 subplots (6 in total)
    fig, axs = plt.subplots(2, 3, figsize=(15, 8))

    # Loop over the target variables (assuming 5 here)
    for i in range(5):
        # Calculate the R² score for validation and test sets
        r2_val = r2_score(Y_validation[:, i], val_pred[:, i])

        if show_test:
            r2_test = r2_score(Y_test[:, i], test_pred[:, i])

        # Plot actual vs predicted for validation (and optionally test) sets
        ax = axs[i // 3, i % 3]  # Subplot position

        if show_test:
            # Plot test data if the flag is set to True
            ax.scatter(Y_test[:, i], test_pred[:, i], s=3, color="#32a895", alpha=0.2, label='Test')

            # Plot the ideal 1:1 line (using ranges from test)
            ax.plot([min(test_pred[:, i]), max(test_pred[:, i])],
                    [min(test_pred[:, i]), max(test_pred[:, i])], color='#f73946', linestyle='--', label='Target Line')
        else:
            # Plot the ideal 1:1 line (using ranges from val)
            ax.plot([min(val_pred[:, i]), max(val_pred[:, i])],
                    [min(val_pred[:, i]), max(val_pred[:, i])], color='#f73946', linestyle='--', label='Target Line')

        # Plot validation data
        ax.scatter(Y_validation[:, i], val_pred[:, i], s=3, color="black", alpha=0.2, label='Validation')

        # Set labels for the axes
        ax.set_xlabel(f"Actual {names[i]}")
        ax.set_ylabel(f"Predicted {names[i]}")

        # Add the R² scores to the title of each subplot
        if show_test:
            ax.set_title(f"{names[i]} - R² Val: {r2_val:.2f}, R² Test: {r2_test:.2f}")
        else:
            ax.set_title(f"{names[i]} - R² Val: {r2_val:.2f}")

        # Remove the top and right borders for a cleaner look
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

    # Add legend in the last subplot (bottom right, unfilled one)
    axs[1, 2].axis('off')  # Turn off the empty subplot
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from previous plots

    # Make points in the legend larger by adjusting 'scatterpoints' and 'markerscale'
    axs[1, 2].legend(handles, labels, loc='center', scatterpoints=3, markerscale=4, handletextpad=1.5)

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

